In [1]:
# Transportation

# Parking Area

# Suppliers
import requests 
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.yellowpages.vn/cls/157610/xay-xat-nha-may-xay-xat-lua-gao.html'
params = {
    'page':1
}
responses = []
for i in range(1, 4):
    params['page'] = i
    response = requests.get(url, params)
    print(response)
    responses.append(response.content)

<Response [200]>
<Response [200]>
<Response [200]>


In [3]:
# name, url
companies = []
links = []

for response in responses:
    bs = BeautifulSoup(response, 'html.parser')
    names_h2 = bs.find_all('h2', {'class':'fs-5 pb-0 text-capitalize'})
    names = [str(BeautifulSoup(str(name), 'html.parser').find('a').get_text()) for name in names_h2]
    
    lks = [str(BeautifulSoup(str(name), 'html.parser').find('a').get('href')) for name in names_h2]
    companies.extend(names)
    links.extend(lks)

In [4]:
df = pd.DataFrame({
    'company_name':companies,
    'links':links
})
df

,company_name,links
0,Công Ty TNHH MTV Minh Tâm Phát,https://www.yellowpages.vn/lgs/1187832396/cong...
1,Công Ty Cổ Phần Sản Xuất Thương Mại Lúa Vàng Việt,https://www.yellowpages.vn/lgs/1187951340/cong...
2,Công Ty TNHH Chế Biến Nông Sản Song Long,https://www.yellowpages.vn/lgs/1187841054/cong...
3,Công ty TNHH Ngọc Ngọc Hà,https://www.yellowpages.vn/lgs/1187811495/cong...
4,Công Ty Cổ Phần Hoàng Minh Nhật,https://www.yellowpages.vn/lgs/1187958873/cong...
...,...,...
111,Phước Sinh - Công Ty TNHH Thương Mại Dịch Vụ P...,https://www.yellowpages.vn/lgs/1010550/phuoc-s...
112,Nhà Máy Xát Xay Gạo Dung Chiếm,https://www.yellowpages.vn/lgs/1187855538/nha-...
113,Doanh Nghiệp Tư Nhân Tài Lợi,https://www.yellowpages.vn/lgs/705432/doanh-ng...
114,Doanh Nghiệp Tư Nhân Bảo Vân,https://www.yellowpages.vn/lgs/705006/doanh-ng...


In [5]:
# Suppliers
# Name x, address x, location ggmap, phone x, representative x, 
# email x, description, sector, products and service, masothue, 
# year of establishment, scale, capacity
import regex as re
from tqdm import tqdm
companies_data = []

def clean_text(text):
    return re.sub(r'[\n\t\r]', '', text)

for com, url in tqdm(zip(companies, links)):
    try:
        name = com
        response = requests.get(url)
        bs = BeautifulSoup(response.content, 'html.parser')
    
        address = bs.find('p', {'class':'m-0 pb-2'}).get_text().strip()
        # gg loc map
        
        lh = str(bs.find('div', {'class':'h-auto w-100 pt-2 pb-3'}))
        lh = BeautifulSoup(str(lh), 'html.parser')
        lh_lines = lh.find_all('span', {'class':'fw-semibold'})
        rep = lh_lines[0].get_text().strip()
        phone=lh_lines[1].get_text().strip()
        email = lh.find('a', {'id': 'email_lienhe'}).get_text().strip()
    
        introcom = str(bs.find_all('div', {'class':'h-auto border shadow-sm p-3 rounded-4 pb-4 pt-4 yp_div_listing_left'})[0])
        introcom = BeautifulSoup(introcom, 'html.parser')
        desc = introcom.find('p', {'class':'m-0 pt-2 lh-lg'}).get_text()
    
        sectors = BeautifulSoup(str(introcom.find_all('div', {'class':'yp_div_nganh_thitruong'})), 'html.parser')
        sectors = sectors.find_all('p')
        sectors = [st.get_text().strip() for st in sectors]
        sector = sectors[0]
        product_and_services=';'.join(sectors)
        market=sectors[-1]
    
        hoso=introcom.find_all('div', {'class':'hoso_right'})
        hoso=[hs.get_text().strip() for hs in hoso]
        mst = hoso[3]
        year=hoso[4]
        scale=hoso[5]
        capacity= 100
        price
    
        line ={
            'CompanyName':name,
            'Address':address,
            'phone':phone,
            'representative':rep,
            'email':email,
            'description':clean_text(desc),
            'sector':sector,
            'ProductAndService':product_and_services,
            'Market':market,
            'Masothue':mst,
            'Year':year,
            'Scale':scale,
            'Capacity':capacity,
            'price':
        }
        # print(line)
        companies_data.append(line)
    except IndexError as e:
        print(e)

4it [00:04,  1.13s/it]


KeyboardInterrupt: 

In [20]:
companies_data

[{'CompanyName': 'Công Ty TNHH MTV Minh Tâm Phát',
  'Address': '261 KP. 3, QL. 1A, TT. Bến Lức, H. Bến Lức, Long An',
  'phone': '0913 958 373',
  'representative': 'Mr. Phạm Hoàng Phát',
  'email': 'minhtamphat@gmail.com',
  'description': 'Công Ty TNHH MTV Minh Tâm Phát thành lập và hoạt động từ năm 2004 chuyên: + Xay xát các loại gạo đặc sản và thông dụng + Cung cấp gạo cho bếp ăn công nghiệp - trường học Sản phẩm được sản xuất đạt chuẩn An Toàn Vệ Sinh Thực Phẩm bộ NN&PTNT Long An cấp.',
  'sector': 'Xay Xát - Nhà Máy Xay Xát Lúa Gạo',
  'ProductAndService': 'Xay Xát - Nhà Máy Xay Xát Lúa Gạo;Gạo - Công Ty Và Đại Lý Gạo;Nhà sản xuất, Công ty Thương mại;Toàn quốc',
  'Market': 'Toàn quốc',
  'Masothue': '1100523365',
  'Year': '2004',
  'Scale': 'Từ 11  - 50 người',
  'Capacity': 189.88346993936392,
  'Price': 3124.3107264463906},
 {'CompanyName': 'Công Ty TNHH Chế Biến Nông Sản Song Long',
  'Address': 'Thôn 3, Bắc Ruộng, Tánh Linh, Bình Thuận',
  'phone': '0971 173 168',
  'repre

In [5]:
import requests
from bs4 import BeautifulSoup
import regex as re
from tqdm import tqdm
import random

def clean_text(text):
    return re.sub(r'[\n\t\r]', '', text)

companies_data = []
cid = 0

for com, url in tqdm(zip(companies, links)):
    
    try:
        name = com
        response = requests.get(url)
        bs = BeautifulSoup(response.content, 'html.parser')
        
        # Extract Address
        address = bs.select_one('p.m-0.pb-2')
        address = address.get_text().strip() if address else 'No address available'
        
        # Extract contact information block
        lh = bs.select_one('div.h-auto.w-100.pt-2.pb-3')
        if lh:
            lh = BeautifulSoup(str(lh), 'html.parser')
            lh_lines = lh.find_all('span', {'class': 'fw-semibold'})
            
            rep = lh_lines[0].get_text().strip() if len(lh_lines) > 0 else 'No representative'
            phone = lh_lines[1].get_text().strip() if len(lh_lines) > 1 else 'No phone'
            email = lh.select_one('a#email_lienhe')
            email = email.get_text().strip() if email else 'No email'
        else:
            rep, phone, email = 'No representative', 'No phone', 'No email'

        # Extract description, ensure introcom is not None
        introcom = bs.select_one('div.h-auto.border.shadow-sm.p-3.rounded-4.pb-4.pt-4')
        desc = introcom.select_one('p.m-0.pt-2.lh-lg').get_text().strip() if introcom and introcom.select_one('p.m-0.pt-2.lh-lg') else 'No description'

        # Extract sectors and market
        sectors = introcom.select('div.yp_div_nganh_thitruong p') if introcom else []
        sectors = [s.get_text().strip() for s in sectors]
        sector = sectors[0] if sectors else 'No sector'
        product_and_services = ';'.join(sectors)
        market = sectors[-1] if sectors else 'No market'
        
        # Extract additional company info
        hoso = introcom.select('div.hoso_right') if introcom else []
        hoso = [hs.get_text().strip() for hs in hoso]
        mst = hoso[3] if len(hoso) > 3 else 'No tax code'
        year = hoso[4] if len(hoso) > 4 else 'No year'
        scale = hoso[5] if len(hoso) > 5 else 'No scale'
        
        # Add random Capacity between 500 and 2000
        capacity = random.uniform(0.5, 2) * 100

        price = random.uniform(0.5, 2.5) * 1000
        cid = cid+1
        # Construct the company data entry
        line = {
            'id':f'company_{cid}',
            'CompanyName': name,
            'Address': address,
            'phone': phone,
            'representative': rep,
            'email': email,
            'description': clean_text(desc),
            'sector': sector,
            'ProductAndService': product_and_services,
            'Market': market,
            'Masothue': mst,
            'Year': year,
            'Scale': scale,
            'Capacity': capacity,   # Random capacity value
            'Price':price
        }
        companies_data.append(line)
    
    except (IndexError, AttributeError) as e:
        print(f"Error for {name} ({url}): {e}")
        # Handle specific case for IndexError or AttributeError by re-parsing or applying fallback
        sectors = bs.find_all('p')
        sectors = [s.get_text().strip() for s in sectors if 'sector' in s.get_text().lower()]
        sector = sectors[0] if sectors else 'Fallback sector'
        product_and_services = ';'.join(sectors)
        cid = cid+1
        # Add fallback with random capacity
        companies_data.append({
            'id':f'company_{cid}',
            'CompanyName': name,
            'Address': 'Fallback Address',
            'phone': 'Fallback phone',
            'representative': 'Fallback representative',
            'email': 'Fallback email',
            'description': 'Fallback description',
            'sector': sector,
            'ProductAndService': product_and_services,
            'Market': 'Fallback market',
            'Masothue': 'Fallback MST',
            'Year': 'Fallback Year',
            'Scale': 'Fallback Scale',
            'Capacity': random.uniform(0.5, 2) * 100  # Random capacity in fallback as well
        })


116it [02:11,  1.13s/it]


In [6]:
len(companies_data)

116

In [8]:
pd.DataFrame(companies_data).to_csv('suppliers.csv', encoding='utf8', index=False)

In [9]:
parking_areas = [
    {
        'id':'parking_1',
        "name": "Bãi Đỗ Xe VSIP 1",
        "address": "Đường số 2, Khu Công Nghiệp VSIP I, Thủ Dầu Một, Bình Dương"
    },
    {
        'id':'parking_2',
        "name": "Bãi Đỗ Xe Tân Bình",
        "address": "246 Đường D1, Khu Công Nghiệp VSIP I, Thủ Dầu Một, Bình Dương"
    },
    {
        'id':'parking_3',
        "name": "Bãi Đỗ Xe Mỹ Phước",
        "address": "Khu Công Nghiệp Mỹ Phước 1, Bến Cát, Bình Dương"
    },
    {
        'id':'parking_4',
        "name": "Bãi Đỗ Xe Mỹ Phước 2",
        "address": "Khu Công Nghiệp Mỹ Phước 2, Bến Cát, Bình Dương"
    },
    {
        'id':'parking_5',
        "name": "Bãi Đỗ Xe Trảng Bàng",
        "address": "Đường Nguyễn Thái Học, Trảng Bàng, Tây Ninh"
    },
    {
        'id':'parking_6',
        "name": "Bãi Đỗ Xe Lê Lợi",
        "address": "100 Đường Lê Lợi, Trảng Bàng, Tây Ninh"
    },
    {
        'id':'parking_7',
        "name": "Bãi Đỗ Xe Ngã Ba Vũng Tàu",
        "address": "Quốc lộ 51, Ngã Ba Vũng Tàu, Long Thành, Đồng Nai"
    },
    {
        'id':'parking_8',
        "name": "Bãi Đỗ Xe Thế Giới Di Động",
        "address": "64 Quốc lộ 51, Long Thành, Đồng Nai"
    },
    {
        'id':'parking_9',
        "name": "Bãi Đỗ Xe Cần Thơ",
        "address": "Đường Trần Văn Khéo, P. An Khánh, Q. Ninh Kiều, TP. Cần Thơ"
    },
    {
        'id':'parking_10',
        "name": "Bãi Đỗ Xe Chợ Nổi Cái Răng",
        "address": "Khu vực chợ nổi, Cái Răng, TP. Cần Thơ"
    },
    {
        'id':'parking_11',
        "name": "Bãi Đỗ Xe Mỹ Tho",
        "address": "Đường 30 Tháng 4, P. 1, TP. Mỹ Tho, Tiền Giang"
    },
    {
        'id':'parking_12',
        "name": "Bãi Đỗ Xe Bến Tre",
        "address": "Đường Nguyễn Thị Định, TP. Bến Tre"
    },
    {
        'id':'parking_13',
        "name": "Bãi Đỗ Xe Trà Vinh",
        "address": "Đường Lê Lợi, P. 1, TP. Trà Vinh"
    },
    {
        'id':'parking_14',
        "name": "Bãi Đỗ Xe Sóc Trăng",
        "address": "Đường Phú Lợi, P. 2, TP. Sóc Trăng"
    },
    {
        'id':'parking_15',
        "name": "Bãi Đỗ Xe Long An",
        "address": "Khu vực Bến Xe Long An, TP. Tân An, Long An"
    },
    {
        'id':'parking_16',
        "name": "Bãi Đỗ Xe An Giang",
        "address": "Khu vực Bến Xe An Giang, TP. Long Xuyên"
    },
    {
        'id':'parking_17',
        "name": "Bãi Đỗ Xe Kiên Giang",
        "address": "Khu vực Bến Xe Kiên Giang, TP. Rạch Giá"
    },
    {
        'id':'parking_18',
        "name": "Bãi Đỗ Xe Vĩnh Long",
        "address": "Khu vực Bến Xe Vĩnh Long, TP. Vĩnh Long"
    }
]


pd.DataFrame(parking_areas).to_csv('./parkings.csv', encoding='utf8')

In [10]:
# Transportations
# registered_number, transport name, capacity, kilometer driven
import random

random.seed(2003)
def rand_platenum():
    return random.randint(1000, 99999)

def rand_kmdriven(cap):
    if cap <10:
        return random.randint(400, 800)
    elif cap <20:
        return random.randint(600, 1200)
    else:
        return random.randint(500, 1000)

mau_xe_van_chuyen = [
    {"name": "Hyundai HD120SL", "capacity_tons": 8},
    {"name": "Isuzu FVM34T", "capacity_tons": 15},
    {"name": "Hino FG8JT7A", "capacity_tons": 15},
    {"name": "Thaco Auman C240", "capacity_tons": 17},
    {"name": "Hyundai Xcient", "capacity_tons": 18},
    {"name": "Dongfeng B180", "capacity_tons": 9},
    {"name": "Fuso FI170", "capacity_tons": 8},
    {"name": "Jac HFC1383K", "capacity_tons": 14},
    {"name": "Howo T5G", "capacity_tons": 20},
    {"name": "Tata Ultra 1014", "capacity_tons": 8}
]

registered_header = [
    "63",  # Tiền Giang
    "64",  # Vĩnh Long
    "65",  # Cần Thơ (thành phố trực thuộc Trung ương)
    "66",  # Đồng Tháp
    "67",  # An Giang
    "68",   # Kiên Giang
    "50",  # TP. Hồ Chí Minh
    "51",  # TP. Hồ Chí Minh
    "52",  # TP. Hồ Chí Minh
    "53",  # TP. Hồ Chí Minh
    "54",  # TP. Hồ Chí Minh
    "55",  # TP. Hồ Chí Minh
    "56",  # TP. Hồ Chí Minh
    "57",  # TP. Hồ Chí Minh
    "58",  # TP. Hồ Chí Minh
    "59"   # TP. Hồ Chí Minh
]

vehicles = []

for i in range(20):
    registered_num=registered_header[random.randint(0,len(registered_header)-1)]+'LD '+str(rand_platenum())
    veh = mau_xe_van_chuyen[random.randint(0,len(mau_xe_van_chuyen)-1)]
    name = veh['name']
    capacity = veh['capacity_tons']
    line = {
        'id':f'vehicle_{i}',
        'registered_num':registered_num,
        'vehicle_name':name,
        'capacity':capacity,
        'kilometer_driven':rand_kmdriven(capacity),
    }
    vehicles.append(line)

In [11]:
pd.DataFrame(vehicles).to_csv('vehicles.csv', index=False)

In [36]:
for com in companies_data:
    print(com['Address'])

261 KP. 3, QL. 1A, TT. Bến Lức, H. Bến Lức, Long An
Thôn 3, Bắc Ruộng, Tánh Linh, Bình Thuận
785-787 Cách Mạng Tháng tám, Khu Phố 2, Phường 3, Thành Phố Tây Ninh, Tây Ninh
Số 15, Tổ 1, Ấp An Long, Xã An Thạnh Trung, Huyện Chợ Mới, An Giang
781 ấp 6B, X. Tân Hoà Châu Thành A, Hậu Giang
59 Tạ Uyên, P. 15, Q. 5, TP. Hồ Chí Minh (TPHCM)
Số 18/2, Đường tỉnh lộ 922, ấp Thới Khánh A, xã Tân Thạnh, H. Thới Lai, TP. Cần Thơ
Cụm CN Gò Đá Trắng, Thị Trấn Đập Đá, An Nhơn, Bình Định
Tổ 1, Ấp Tây, X. Long phước, TP. Bà Rịa, Bà Rịa-Vũng Tàu
ấp Bình Hiệp B, X. Bình Thạnh Trung, H. Lấp Vò, Đồng Tháp
ấp Cầu, X. Khánh Hậu, TX. Tân An, Long An
ấp Thới Thuận B, TT. Thới Lai, H. Cờ Đỏ, Tp. Cần Thơ
ấp Phúc Lộc, X. Trung Nhứt,H. Thốt Nốt, Tp. Cần Thơ
X. Tân Đông,ấp 1, H. Thạnh Hóa, Long An
36B Đinh Tiên Hoàng, P. 8, Vĩnh Long
Thôn 4, X. Hồng Sơn,Q. Hàm Thuận Bắc, Bình Thuận
5 Đường 834, Khu Phố Thanh Xuân,P. 5, TX. Tân An, Long An
ấp 3, X. Nhơn Hòa Lập, H. Tân Thạnh, Long An
K4 Dốc Binh Kiều, TT. Mộc Hóa, H. 

In [6]:
destinations = [
    '5/219, Đ. Thủ Khoa Huân/Tổ 4A Đ.Đ1, Thuận Giao, Thuận An, Bình Dương 75000',
    'B12 KDC Nam Thịnh, KP. Bình Đường 3, P. An Bình, TX. Dĩ An, Bình Dương',
    'Số 52/4 Đường DT 743, Khu Phố Đông Tân, Phường Dĩ An, Thành Phố Dĩ An, Bình Dương',
    'T13/17M Đường D10, Tổ 13, KP. Bình Thuận 2, P. Thuận Giao, TP. Thuận An, Bình Dương',
    'Số 02 Lý Thường Kiệt, Khu Phố Đông Tân, Phường Dĩ An, Thành Phố Dĩ An, Bình Dương'
]
df = pd.DataFrame(columns=['address'], data=destinations)
df.to_csv('destination_v2.csv', encoding='utf8', index=False)
df

,address
0,"5/219, Đ. Thủ Khoa Huân/Tổ 4A Đ.Đ1, Thuận Giao..."
1,"B12 KDC Nam Thịnh, KP. Bình Đường 3, P. An Bìn..."
2,"Số 52/4 Đường DT 743, Khu Phố Đông Tân, Phường..."
3,"T13/17M Đường D10, Tổ 13, KP. Bình Thuận 2, P...."
4,"Số 02 Lý Thường Kiệt, Khu Phố Đông Tân, Phường..."
